In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import json
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.optim as optim
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from transformers import ElectraTokenizer, ElectraForSequenceClassification

In [ ]:
# Load the datasets
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

train_data = load_dataset('/content/drive/MyDrive/DAP/projekat/ELECTRA/Emotion Detection/train.jsonl')
validation_data = load_dataset('/content/drive/MyDrive/DAP/projekat/ELECTRA/Emotion Detection/validation.jsonl')
test_data = load_dataset('/content/drive/MyDrive/DAP/projekat/ELECTRA/Emotion Detection/test.jsonl')

In [ ]:
def text_extract (data):
    text, label = [], []
    for idx in range(len(data)):
        text.append(data[idx]['text'])
        label.append(data[idx]['label'])
    return text, label

train_texts, train_labels= text_extract (train_data)

dev_texts, dev_labels= text_extract (validation_data)

test_texts, test_labels= text_extract (test_data)

In [ ]:
# Load the pre-trained BERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

In [ ]:
# Tokenize the input texts
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_dev_texts = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_test_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

# Convert the labels to tensor
train_labels = torch.tensor(train_labels)

dev_labels = torch.tensor(dev_labels)

test_labels = torch.tensor(test_labels)

In [ ]:
# Define hyperparameters
learning_rate = 8.1e-3
batch_size = 32
weight_decay = 3e-4
num_epochs = 100

In [ ]:
# Define hyperparameters
#learning_rate = 2e-5
#batch_size = 32
#weight_decay = 1e-4
#num_epochs = 5

In [ ]:
# Create TensorDatasets for train, dev, and test sets
train_dataset = TensorDataset(tokenized_train_texts['input_ids'], tokenized_train_texts['attention_mask'], train_labels)

dev_dataset = TensorDataset(tokenized_dev_texts['input_ids'], tokenized_dev_texts['attention_mask'], dev_labels)

test_dataset = TensorDataset(tokenized_test_texts['input_ids'], tokenized_test_texts['attention_mask'], test_labels)


# Create DataLoaders for train, dev, and test sets
train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)

dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)

test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)

In [ ]:
# Load the pre-trained BERT model for sequence classification
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels= 6)

model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=6)

# Freeze all layers except the classification layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the optimizer for training the softmax layer
optimizer = optim.Adam(model.classifier.parameters(), lr= learning_rate, weight_decay= weight_decay)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer= optimizer,
    num_warmup_steps= 0,
    num_training_steps= num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
# Train the model
best_dev_accuracy = 0.0
best_model_state_dict = None
Validation_results= []

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids= input_ids, attention_mask= attention_mask, labels= batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        #### to show progress_bar
        lr_scheduler.step()
        progress_bar.update(1)


    # Validation loop
    model.eval()
    dev_correct = 0
    total_dev = 0
    y_true = []
    y_pred = []
    loss_epoch= []
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids, attention_mask, batch_labels = batch

            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            # Append true labels and predicted labels for later use
            y_true.extend(batch_labels.tolist())
            y_pred.extend(predicted.tolist())

            # Calculate the loss
            loss = criterion(logits, batch_labels)
            loss_epoch.append(loss)

    # Calculate accuracy and F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate the average loss
    loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
    average_loss= np.mean(loss_epoch_np)
    print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')

    Validation_results.append([accuracy, f1, average_loss])

    if accuracy > best_dev_accuracy:
        best_dev_accuracy = accuracy
        # Save the best model (optional)
        best_model_state_dict = model.state_dict()

epoch No. : 0, Devset Accuracy : 0.426, Devset f1_score : 0.32674, Average loss: 1.50919


In [ ]:
# Saving Testset Results
data = {
    'Validation_results': Validation_results,
}
df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/DAP/projekat/ELECTRA/projekat_Validation_results.csv', index= False)

In [ ]:
# Load the best model state dict
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)

    # Define the directory path to save the model
    save_path = '/content/drive/MyDrive/DAP/projekat/ELECTRA/Model-4-3.pth'

    # Save the model state dictionary and other relevant information
    torch.save({
        'model_state_dict': best_model_state_dict,
        'tokenizer': tokenizer
    }, save_path)

In [ ]:
# Evaluate on the test set
model.eval()
y_true_test = []
y_pred_test = []
loss_epoch= []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        # Append true labels and predicted labels for later use
        y_true_test.extend(batch_labels.tolist())
        y_pred_test.extend(predicted.tolist())

        # Calculate the loss
        loss = criterion(logits, batch_labels)
        loss_epoch.append(loss)

# Calculate accuracy and F1 score for the test set
test_accuracy = accuracy_score(y_true_test, y_pred_test)
test_f1 = f1_score(y_true_test, y_pred_test, average='weighted')

# Calculate the average loss
loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
average_loss= np.mean(loss_epoch_np)

print(f"Testset accuracy: {round(test_accuracy,5)} , Testset F1 score: {round(test_f1,5)}, Average loss: {round(average_loss.tolist(),5)}")
Test_results= [test_accuracy, test_f1, average_loss]

In [ ]:
# Saving Testset Results
data = {
    'Test_results': Test_results
}
df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/DAP/projekat/ELECTRA/projekat_Test_results.csv', index= False)